In [1]:
url = "https://homl.info/shakespeare"

In [2]:
from  tensorflow import keras
fpath = keras.utils.get_file("shakespeare.txt",url)

In [3]:
with open(fpath) as fp:
    s_text = fp.read()

In [4]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(s_text)

In [5]:
max_id = len(tokenizer.word_index)
max_id

39

In [6]:
d_size = tokenizer.document_count
d_size

1115394

In [7]:
import numpy as np
[encoded] = np.array(tokenizer.texts_to_sequences([s_text]))-1

In [151]:
len(encoded)

1115394

In [152]:
t_s = d_size*90//100

In [153]:
import tensorflow as tf
data = tf.data.Dataset.from_tensor_slices(encoded[:t_s])

In [154]:
n_steps = 100
w_l =n_steps+1
dataset = data.window(w_l,shift=1,drop_remainder=True)

In [155]:
# tokenizer.sequences_to_texts([[19,5,8,7,2,0,18,5,2,5,35]])

In [156]:
for i in dataset.take(3):
    print(list(i.as_numpy_iterator()))

[19, 5, 8, 7, 2, 0, 18, 5, 2, 5, 35, 1, 9, 23, 10, 21, 1, 19, 3, 8, 1, 0, 16, 1, 0, 22, 8, 3, 18, 1, 1, 12, 0, 4, 9, 15, 0, 19, 13, 8, 2, 6, 1, 8, 17, 0, 6, 1, 4, 8, 0, 14, 1, 0, 7, 22, 1, 4, 24, 26, 10, 10, 4, 11, 11, 23, 10, 7, 22, 1, 4, 24, 17, 0, 7, 22, 1, 4, 24, 26, 10, 10, 19, 5, 8, 7, 2, 0, 18, 5, 2, 5, 35, 1, 9, 23, 10, 15, 3, 13, 0]
[5, 8, 7, 2, 0, 18, 5, 2, 5, 35, 1, 9, 23, 10, 21, 1, 19, 3, 8, 1, 0, 16, 1, 0, 22, 8, 3, 18, 1, 1, 12, 0, 4, 9, 15, 0, 19, 13, 8, 2, 6, 1, 8, 17, 0, 6, 1, 4, 8, 0, 14, 1, 0, 7, 22, 1, 4, 24, 26, 10, 10, 4, 11, 11, 23, 10, 7, 22, 1, 4, 24, 17, 0, 7, 22, 1, 4, 24, 26, 10, 10, 19, 5, 8, 7, 2, 0, 18, 5, 2, 5, 35, 1, 9, 23, 10, 15, 3, 13, 0, 4]
[8, 7, 2, 0, 18, 5, 2, 5, 35, 1, 9, 23, 10, 21, 1, 19, 3, 8, 1, 0, 16, 1, 0, 22, 8, 3, 18, 1, 1, 12, 0, 4, 9, 15, 0, 19, 13, 8, 2, 6, 1, 8, 17, 0, 6, 1, 4, 8, 0, 14, 1, 0, 7, 22, 1, 4, 24, 26, 10, 10, 4, 11, 11, 23, 10, 7, 22, 1, 4, 24, 17, 0, 7, 22, 1, 4, 24, 26, 10, 10, 19, 5, 8, 7, 2, 0, 18, 5, 2, 5, 35, 1, 9

In [157]:
dflat = dataset.flat_map(lambda window:window.batch(w_l))

In [158]:
for i in dflat.take(3):
    print(i[90:])

tf.Tensor([ 2  5 35  1  9 23 10 15  3 13  0], shape=(11,), dtype=int32)
tf.Tensor([ 5 35  1  9 23 10 15  3 13  0  4], shape=(11,), dtype=int32)
tf.Tensor([35  1  9 23 10 15  3 13  0  4  8], shape=(11,), dtype=int32)


In [159]:
batch_size = 32
dflat = dflat.shuffle(1000).batch(batch_size)

In [160]:
dflat = dflat.map(lambda windows:(windows[:,:-1],windows[:,1:]))

In [161]:
for i in dflat.take(2):
    print(i)

(<tf.Tensor: shape=(32, 100), dtype=int32, numpy=
array([[ 3,  9,  1, ..., 18,  5,  2],
       [15,  5,  1, ..., 21, 13,  2],
       [13,  2,  6, ..., 16,  6,  3],
       ...,
       [ 0,  5,  2, ...,  1, 12,  0],
       [ 0, 22,  1, ...,  3, 16,  9],
       [ 4,  9, 18, ...,  3,  8,  0]])>, <tf.Tensor: shape=(32, 100), dtype=int32, numpy=
array([[ 9,  1, 23, ...,  5,  2,  5],
       [ 5,  1, 11, ..., 13,  2,  0],
       [ 2,  6,  3, ...,  6,  3, 11],
       ...,
       [ 5,  2,  0, ..., 12,  0, 22],
       [22,  1,  3, ..., 16,  9,  0],
       [ 9, 18,  1, ...,  8,  0,  2]])>)
(<tf.Tensor: shape=(32, 100), dtype=int32, numpy=
array([[26,  0,  8, ..., 24,  9,  3],
       [ 0,  4, 16, ...,  2,  6,  1],
       [ 1, 14,  0, ..., 20,  1,  8],
       ...,
       [ 3,  0, 22, ..., 24,  1,  7],
       [ 3,  0, 14, ...,  0, 18,  5],
       [ 1,  7,  3, ...,  4,  5, 13]])>, <tf.Tensor: shape=(32, 100), dtype=int32, numpy=
array([[ 0,  8,  1, ...,  9,  3, 16],
       [ 4, 16,  4, ...,  6,  1,  0

In [144]:
# tf.one_hot([1,2,4],depth=4)

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

In [162]:
dflat = dflat.map(lambda X_b,Y_b: (tf.one_hot(X_b,depth=max_id),Y_b))

In [163]:
for i in dflat.take(2):
    print(i)    

(<tf.Tensor: shape=(32, 100, 39), dtype=float32, numpy=
array([[[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0.

In [165]:
dflat = dflat.prefetch(1)

In [167]:
for X_batch, Y_batch in dflat.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


In [169]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dflat, epochs=3)

Epoch 1/3
31368/31368 [==============================] - 3253s 104ms/step - loss: 1.6328
Epoch 2/3
31368/31368 [==============================] - 5425s 173ms/step - loss: 1.5685
Epoch 3/3
31368/31368 [==============================] - 3398s 108ms/step - loss: 1.5487


In [186]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [191]:
X_new = preprocess("How are you")
Y_pred = np.argmax(model(X_new), axis=-1)


In [206]:
X_new = preprocess(["How are yo"])
#Y_pred = model.predict_classes(X_new)
Y_pred = np.argmax(model(X_new), axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

'u'

In [217]:
model.predict(X_new)[0,-1:,:]

array([[4.0459199e-04, 3.2932534e-05, 1.4920449e-03, 1.9692015e-05,
        1.8722432e-04, 1.7949014e-05, 2.9811654e-05, 2.0848440e-05,
        4.8933463e-04, 1.7941782e-04, 1.8850882e-05, 2.0682439e-06,
        1.1006093e-05, 9.9567324e-01, 3.3501750e-05, 9.6338154e-05,
        3.0981025e-04, 1.6488614e-04, 5.7076562e-05, 3.9632064e-06,
        9.1000911e-06, 1.6556985e-06, 6.1151047e-07, 2.7117196e-05,
        1.2799128e-06, 3.5007508e-04, 1.2869040e-04, 5.3823887e-06,
        1.1158750e-05, 2.1245422e-04, 3.4638349e-06, 2.4673222e-06,
        4.2870617e-08, 4.8004026e-08, 1.2040691e-07, 1.6494181e-06,
        2.2848929e-09, 5.8693522e-11, 2.8754110e-11]], dtype=float32)

In [215]:
model(X_new)

<tf.Tensor: shape=(1, 10, 39), dtype=float32, numpy=
array([[[3.76836918e-02, 4.27791715e-01, 9.95732471e-03, 1.01372473e-01,
         2.13540092e-01, 9.34411734e-02, 2.11075624e-03, 6.65976433e-03,
         5.99742169e-03, 2.27317121e-03, 3.56663438e-03, 5.39182685e-03,
         2.90621724e-03, 1.54922921e-02, 1.48994348e-03, 2.98371091e-02,
         9.17096331e-04, 1.53627349e-02, 6.64529915e-04, 1.94426475e-03,
         1.20241055e-03, 1.88748367e-04, 7.27309962e-04, 4.87992307e-03,
         7.58843904e-04, 4.71356208e-04, 5.29873371e-03, 3.10155726e-03,
         8.84147827e-04, 1.94265228e-03, 1.64149306e-03, 3.06330592e-04,
         4.59894582e-05, 8.91273012e-05, 1.66856280e-05, 4.32945817e-05,
         9.30368174e-07, 9.28278752e-08, 8.31766513e-08],
        [2.36906242e-02, 1.23658115e-02, 1.87193379e-02, 2.74189543e-02,
         5.80734946e-03, 3.09599284e-03, 8.31784506e-04, 2.87892856e-02,
         2.13397294e-01, 8.90928358e-02, 1.29531277e-03, 7.58156255e-02,
         4.65

In [219]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:t_s])

In [222]:
list(dataset.take(10).as_numpy_iterator())

[19, 5, 8, 7, 2, 0, 18, 5, 2, 5]

In [223]:
dataset = dataset.window(w_l, shift=n_steps, drop_remainder=True)

In [228]:
for i in dataset.take(3):
    print(list(i.as_numpy_iterator()))

[19, 5, 8, 7, 2, 0, 18, 5, 2, 5, 35, 1, 9, 23, 10, 21, 1, 19, 3, 8, 1, 0, 16, 1, 0, 22, 8, 3, 18, 1, 1, 12, 0, 4, 9, 15, 0, 19, 13, 8, 2, 6, 1, 8, 17, 0, 6, 1, 4, 8, 0, 14, 1, 0, 7, 22, 1, 4, 24, 26, 10, 10, 4, 11, 11, 23, 10, 7, 22, 1, 4, 24, 17, 0, 7, 22, 1, 4, 24, 26, 10, 10, 19, 5, 8, 7, 2, 0, 18, 5, 2, 5, 35, 1, 9, 23, 10, 15, 3, 13, 0]
[0, 4, 8, 1, 0, 4, 11, 11, 0, 8, 1, 7, 3, 11, 25, 1, 12, 0, 8, 4, 2, 6, 1, 8, 0, 2, 3, 0, 12, 5, 1, 0, 2, 6, 4, 9, 0, 2, 3, 0, 19, 4, 14, 5, 7, 6, 29, 10, 10, 4, 11, 11, 23, 10, 8, 1, 7, 3, 11, 25, 1, 12, 26, 0, 8, 1, 7, 3, 11, 25, 1, 12, 26, 10, 10, 19, 5, 8, 7, 2, 0, 18, 5, 2, 5, 35, 1, 9, 23, 10, 19, 5, 8, 7, 2, 17, 0, 15, 3, 13, 0]
[0, 24, 9, 3, 16, 0, 18, 4, 5, 13, 7, 0, 14, 4, 8, 18, 5, 13, 7, 0, 5, 7, 0, 18, 6, 5, 1, 19, 0, 1, 9, 1, 14, 15, 0, 2, 3, 0, 2, 6, 1, 0, 22, 1, 3, 22, 11, 1, 26, 10, 10, 4, 11, 11, 23, 10, 16, 1, 0, 24, 9, 3, 16, 27, 2, 17, 0, 16, 1, 0, 24, 9, 3, 16, 27, 2, 26, 10, 10, 19, 5, 8, 7, 2, 0, 18, 5, 2, 5, 35, 1, 9, 23, 1

In [230]:
dataset = dataset.flat_map(lambda window: window.batch(w_l))


In [231]:
for i in dataset.take(3):
    print(i)

tf.Tensor(
[19  5  8  7  2  0 18  5  2  5 35  1  9 23 10 21  1 19  3  8  1  0 16  1
  0 22  8  3 18  1  1 12  0  4  9 15  0 19 13  8  2  6  1  8 17  0  6  1
  4  8  0 14  1  0  7 22  1  4 24 26 10 10  4 11 11 23 10  7 22  1  4 24
 17  0  7 22  1  4 24 26 10 10 19  5  8  7  2  0 18  5  2  5 35  1  9 23
 10 15  3 13  0], shape=(101,), dtype=int32)
tf.Tensor(
[ 0  4  8  1  0  4 11 11  0  8  1  7  3 11 25  1 12  0  8  4  2  6  1  8
  0  2  3  0 12  5  1  0  2  6  4  9  0  2  3  0 19  4 14  5  7  6 29 10
 10  4 11 11 23 10  8  1  7  3 11 25  1 12 26  0  8  1  7  3 11 25  1 12
 26 10 10 19  5  8  7  2  0 18  5  2  5 35  1  9 23 10 19  5  8  7  2 17
  0 15  3 13  0], shape=(101,), dtype=int32)
tf.Tensor(
[ 0 24  9  3 16  0 18  4  5 13  7  0 14  4  8 18  5 13  7  0  5  7  0 18
  6  5  1 19  0  1  9  1 14 15  0  2  3  0  2  6  1  0 22  1  3 22 11  1
 26 10 10  4 11 11 23 10 16  1  0 24  9  3 16 27  2 17  0 16  1  0 24  9
  3 16 27  2 26 10 10 19  5  8  7  2  0 18  5  2  5 35  1  9 23 10 11  1
 

In [238]:
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()

17473536/17464789 [==============================] - 1s 0us/step


In [247]:
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [250]:
y_train.shape

(25000,)

In [251]:
word_index = keras.datasets.imdb.get_word_index()

1654784/1641221 [==============================] - 0s 0us/step


In [255]:
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

'<sos> this film was just brilliant casting location scenery story'

In [258]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

C:\Users\Pankhuri\.conda\envs\tf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dl Completed...: 100%|██████████| 1/1 [00:15<00:00, 15.47s/ url]


Dataset imdb_reviews downloaded and prepared to C:\Users\Pankhuri\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [317]:
train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples

In [356]:
# for X_batch, y_batch in datasets["train"].batch(3).take(1):
#     print(X_batch)

In [355]:
for X_batch, y_batch in datasets["train"].batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print("Review:", review.decode("utf-8"), "...")
        print("Label:", label, "= Positive" if label else "= Negative")
        print()

Review: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it. ...
Label: 0 = Negative

Review: I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film w

In [354]:
# X_batch.numpy().shape

In [353]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [351]:
# preprocess(X_batch, y_batch)[0][0]

In [352]:
# X_batch = tf.strings.substr(X_batch, 0, 5)
# X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
# X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
# X_batch = tf.strings.split(X_batch)
# X_batch.to_tensor(default_value=b"<pad>")

In [357]:
preprocess(X_batch, y_batch)

(<tf.Tensor: shape=(2, 53), dtype=string, numpy=
 array([[b'This', b'was', b'an', b'absolutely', b'terrible', b'movie',
         b"Don't", b'be', b'lured', b'in', b'by', b'Christopher',
         b'Walken', b'or', b'Michael', b'Ironside', b'Both', b'are',
         b'great', b'actors', b'but', b'this', b'must', b'simply', b'be',
         b'their', b'worst', b'role', b'in', b'history', b'Even',
         b'their', b'great', b'acting', b'could', b'not', b'redeem',
         b'this', b"movie's", b'ridiculous', b'storyline', b'This',
         b'movie', b'is', b'an', b'early', b'nineties', b'US',
         b'propaganda', b'pi', b'<pad>', b'<pad>', b'<pad>'],
        [b'I', b'have', b'been', b'known', b'to', b'fall', b'asleep',
         b'during', b'films', b'but', b'this', b'is', b'usually', b'due',
         b'to', b'a', b'combination', b'of', b'things', b'including',
         b'really', b'tired', b'being', b'warm', b'and', b'comfortable',
         b'on', b'the', b'sette', b'and', b'having', b'j

In [358]:
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [359]:
vocabulary.most_common()[:3]

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

In [360]:
len(vocabulary)

53893

In [361]:
vocab_size = 10000
truncated_vocabulary = [word for word, count in vocabulary.most_common()[:vocab_size]]

In [362]:
word_to_id = {word: index for index, word in enumerate(truncated_vocabulary)}
for word in b"This movie was faaaaaantastic".split():
    print(word_to_id.get(word) or vocab_size)

22
12
11
10000


In [372]:
# print(word_to_id.get(b'Thicdcs') or 111)

In [376]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [377]:
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10053]], dtype=int64)>

In [378]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [382]:
for X_batch, y_batch in train_set.take(5):
    print(X_batch.shape)
    print(y_batch.shape)

(32, 60)
(32,)
(32, 60)
(32,)
(32, 63)
(32,)
(32, 61)
(32,)
(32, 63)
(32,)


In [383]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 69s 81ms/step - loss: 0.5357 - accuracy: 0.7249
Epoch 2/5
782/782 [==============================] - 60s 76ms/step - loss: 0.3434 - accuracy: 0.8582
Epoch 3/5
782/782 [==============================] - 63s 81ms/step - loss: 0.1883 - accuracy: 0.9342
Epoch 4/5
782/782 [==============================] - 64s 81ms/step - loss: 0.1436 - accuracy: 0.9492
Epoch 5/5
782/782 [==============================] - 62s 79ms/step - loss: 0.1179 - accuracy: 0.9559


In [ ]:
vocab_size + num_oov_buckets